In [2]:
# Import python packages numpy, matplotlib, and pulp
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pulp

# CEE 201: Linear Programming with Jupyter Notebooks

## Example: A Blending Problem

### Problem Statement

A big pharma company is preparing a batch of COVID-19 vaccines for a small scale experiment. Ingredients, their costs, and availability are presented in the following table:

|Ingredient   | Cost ($/kg)   | Availability   |
|----------------|----------------|----------------|
| Antigen 1   |  4.32 |  30     | 
| Antigen 2  | 2.46  | 20  |
| Antigen 3  | 1.86 | 17|

Their lab experts will make 2 types of vaccines:

* Kids (>40% Antigen 1)
* Adults (>60% Antigen 1)

One vaccine is 0.05 kg. According to FDA regulations, the most antigen 3 we can use in our vaccine is 25%. In addition, we have a contract with a medical provider, and have already purchased 23 kg of Antigen 1, that must go in our vaccines. In our experiment, 350 kids and 500 adult vaccines will be used.

We need to figure out **how to most cost effectively blend our antigens to prepare vaccines for our trial**.

#### Decision Varibles

* $k_1$= antigen 1 in kids vaccine (kg)
* $k_2$= antigen 2 in kids vaccine (kg)
* $k_3$= antigen 3 in kids vaccine (kg)
* $a_1$= antigen 1 in adults vaccine (kg)
* $a_2$= antigen 2 in adults vaccine (kg)
* $a_3$= antigen 3 in adults vaccine (kg)


### Objective Function & Constraints
#### Minimizing total costs
$Z=4.32(k_1+a_1)+2.46(k_2+a_2)+1.86(k_3+a_3)$

#### Problem costraints
* Vaccines kg requirement \
$k_1+k_2+k_3= 350* 0.005$ \
$a_1+a_2+a_3= 500* 0.005$
* Kids and Adults vaccines antigen 1 dosage \
$k_1≥0.4(k_1+k_2+k_3)$ \
$a_1≥0.6(a_1+a_2+a_3)$
* Antigen 3 FDA regulation dosage \
$k_3≤0.25(k_1+k_2+k_3)$ \
$a_3≤0.25(a_1+a_2+a_3)$
* Antigens  availability \
$k_1+a_1≤30$ \
$k_2+a_2≤20$ \
$k_3+a_3≤17$
* Antigen 1 medical providers supply \
$k_1+a_1≥23$

* Production cannot be negative \
$k_1, k_2, k_3 \geq 0$, \
$a_1, a_2, a_3 \geq 0$

## Solution by Python Pulp package

In [6]:
# Instantiate the problem class
model = pulp.LpProblem("Cost minimising blending problem", pulp.LpMinimize)

We have 6 decision variables, we could name them individually but this wouldn’t scale up if we had hundreds/thousands of variables (you don’t want to be entering all of these by hand multiple times).
We’ll create a couple of lists from which we can create tuple indices.

In [7]:
# Construct our decision variable lists
vaccine_types = ['kids', 'adults']
ingredients = ['antigen 1', 'antigen 2', 'antigen 3']

Each of the decision variables have similar characteristics (lower bound of 0, continuous variables). Therefore, we can use PuLP’s LpVariable object’s dictionary functionality, we can provide our tuple indices. These tuples will be keys for the ing_weight dictionary of decision variables.

In [8]:
# Define characteristics
ing_weight = pulp.LpVariable.dicts("weight kg",
                                     ((i, j) for i in vaccine_types for j in ingredients),
                                     lowBound=0,
                                     cat='Continuous')

In [9]:
# Objective Function
model += (
    pulp.lpSum([
        4.32 * ing_weight[(i, 'antigen 1')]
        + 2.46 * ing_weight[(i, 'antigen 2')]
        + 1.86 * ing_weight[(i, 'antigen 3')]
        for i in vaccine_types])
)

We are going to add the constraints of the problem in this step:

In [11]:
# Constraints
# 350 kids and 500 adults vaccines at 0.05 kg
model += pulp.lpSum([ing_weight['kids', j] for j in ingredients]) == 350 * 0.05
model += pulp.lpSum([ing_weight['adults', j] for j in ingredients]) == 500 * 0.05

# Kids has >= 40% antigen 1, adults >= 60% antigen 1
model += ing_weight['kids', 'antigen 1'] >= (
    0.4 * pulp.lpSum([ing_weight['kids', j] for j in ingredients]))

model += ing_weight['adults', 'antigen 1'] >= (
    0.6 * pulp.lpSum([ing_weight['adults', j] for j in ingredients]))

# Vaccines must be <= 25% antigen 3
model += ing_weight['kids', 'antigen 3'] <= (
    0.25 * pulp.lpSum([ing_weight['kids', j] for j in ingredients]))

model += ing_weight['adults', 'antigen 3'] <= (
    0.25 * pulp.lpSum([ing_weight['adults', j] for j in ingredients]))

# We have at most 30 kg of antigen 1, 20 kg of 2 and 17 kg of 3 available
model += pulp.lpSum([ing_weight[i, 'antigen 1'] for i in vaccine_types]) <= 30
model += pulp.lpSum([ing_weight[i, 'antigen 2'] for i in vaccine_types]) <= 20
model += pulp.lpSum([ing_weight[i, 'antigen 3'] for i in vaccine_types]) <= 17

# We have at least 23 kg of antigen 1 to use up
model += pulp.lpSum([ing_weight[i, 'antigen 1'] for i in vaccine_types]) >= 23

In [12]:
# Solve our problem
model.solve()
pulp.LpStatus[model.status]

'Optimal'

In this step we will review the results:

In [22]:
total_cost = pulp.value(model.objective)

print("The total cost in $ for 350 kids and 500 adults vaccines is",format(round(total_cost, 2)))

The total cost in $ for 350 kids and 500 adults vaccines is 140.96


In [23]:
for var in ing_weight:
    var_value = ing_weight[var].varValue
    print ("The weight of {0} in {1} sausages is {2} kg",format(var[1], var[0], var_value))

TypeError: format() takes at most 2 arguments (3 given)